In [1]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde.deepxde as dde
import deepxde.deepxde.backend as bkd
from datasets import makeTesting
from datasets import parallel_solver, diffusion_reaction_solver
import time
from scipy import interpolate

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


In [2]:
if False:
    makeTesting()

In [3]:
def pde(x, y):
    vx = x[0].repeat_interleave(101)[:,None]

    D = 0.01
    k = 0.01
    dy_t = dde.grad.jacobian(y, x[1], j=1)
    dy_xx = dde.grad.hessian(y, x[1], j=0)
    return dy_t - D * dy_xx + k * y**2 - vx

In [4]:
space = dde.data.GRF(1.0, length_scale = 0.1, N= 1000, interp="cubic")
vxs = space.eval_batch(space.random(20), np.linspace(0, 1, 101)[:, None])
uxts = parallel_solver(diffusion_reaction_solver, vxs, num_workers = 6)
grid = uxts[0][0].reshape(101 * 101, -1)
uxts = np.asarray([u for grid, u in uxts]).reshape(-1, 101 * 101)


train_vxs = vxs
train_grid = grid
train_uxts = uxts
print(train_vxs.shape, train_grid.shape, train_uxts.shape)

test_data = np.load("datasets/DF_100_0.1_101_101.npz")
test_vxs = test_data["vxs"].astype(np.float32)
test_grid = test_data["xt"].reshape(-1, 2).astype(np.float32)
test_uxts = test_data["uxts"].reshape(-1, 101 * 101).astype(np.float32)
del test_data
print(test_vxs.shape, test_grid.shape, test_uxts.shape)

(20, 101) (10201, 2) (20, 10201)
(100, 101) (10201, 2) (100, 10201)


In [5]:
data = dde.data.TripleCartesianProd(X_train=(train_vxs, train_grid), y_train=train_uxts, X_test=(test_vxs, test_grid), y_test=test_uxts)

# Net
net = dde.nn.DeepONetCartesianProd(
    [101, 128, 128, 128],
    [2, 128, 128, 128],
    "tanh",
    "Glorot normal",
)

model = dde.Model(data, net)
model.compile("adam", lr=0.0005)
losshistory, train_state = model.train(iterations=5000)
dde.utils.plot_loss_history(losshistory)

Compiling model...
'compile' took 0.000230 s

Training model...

Step      Train loss    Test loss     Test metric
0         [5.28e-01]    [4.65e-01]    []  
1000      [2.71e-02]    [6.97e-02]    []  


KeyboardInterrupt: 

In [6]:

while len(train_vxs) < 200:
    # generate some vxs to test
    x_space = np.linspace(0, 1, 101)
    t_space = np.linspace(0, 1, 101)
    vxs = space.eval_batch(space.random(100), x_space[:, None])
    grid = np.asarray(np.meshgrid(x_space, t_space, indexing = "ij")).transpose([1,2,0]).reshape(101 * 101, -1)
    res = model.predict((vxs, grid), operator = pde)
    res = np.mean(np.abs(res[...,0]), axis = 1)
    topk_index = np.argpartition(res, -20)[-20:]
    topk_vxs = vxs[topk_index]
    uxts = parallel_solver(diffusion_reaction_solver, topk_vxs, num_workers = 6)
    uxts = np.asarray([u for grid, u in uxts]).reshape(-1, 101 * 101)
    train_vxs = np.concatenate([train_vxs, topk_vxs], axis = 0)
    train_uxts = np.concatenate([train_uxts, uxts], axis = 0)
    data = dde.data.TripleCartesianProd(X_train=(train_vxs, train_grid), y_train=train_uxts, X_test=(test_vxs, test_grid), y_test=test_uxts)
    
    model = dde.Model(data, net)
    model.compile("adam", lr=0.0005)
    losshistory, train_state = model.train(iterations=5000)
        


Compiling model...
'compile' took 0.000980 s

Training model...

0         [8.07e-02]    [7.14e-02]    []  
1000      [1.35e-02]    [6.43e-02]    []  


KeyboardInterrupt: 